# ChatGPT Dev Helper

The purpose of this notebook is to provide a quick intro to using the ChatGPT API.

Our examples are:
- Creating a series of related limericks about podcasts
- Generating some fake patients for a GP practice and asking ChatGPT to provide information and next steps for their treatment

## Setup

Import libraries and set static variables

In [ ]:
import openai
import os
import requests

# set API key
openai.api_key = os.environ.get("OPENAI_API_KEY")

COMPLETIONS_MODEL = 'text-davinci-003'
CHATGPT_MODEL = "chat-davinci-003-alpha"

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}
completions_url = 'https://api.openai.com/v1/completions'

In [ ]:
# Simple function that formats a prompt for injection into ChatGPT
def create_prompt(prompt):
    cleaned_prompt = '\n<|im_start|>user\n' + prompt +  '<|im_end|>\n<|im_start|>assistant'
    return cleaned_prompt

# Class for holding ChatGPT context
class ChatGPT():
    
    # Initialises with your OpenAI Key and creates an empty list to hold the dialogue
    def __init__(self,key):
        self.key = key
        # Dialogue stored as tuples consisting of (sender,message)
        self.dialogue = []
        
    
    # Simple function which returns the full conversation so far
    def conversation(self):
        
        if len(self.dialogue) == 0:
            print('No conversation to show')
            
        else:
            conversation = [x[1] for x in self.dialogue]
            return ''.join(conversation)
        
    # Function that takes a prompt and any other model params, and stores user input and assistant feedback in the dialogue    
    def chat(self,prompt,CHATGPT_MODEL="chat-davinci-003-alpha",temperature=0,max_tokens=500):
        
        # Prepare prompt with <im> tags
        prepared_prompt = create_prompt(prompt)
        
        self.dialogue.append(('user',prepared_prompt))
        
        if len(self.dialogue) != 0:
            prepared_prompt = self.conversation() + prepared_prompt
        
        # Builds up JSON data for request
        json_data = {
        "model": CHATGPT_MODEL,
        "prompt": prepared_prompt,
        "temperature": 0,
        "max_tokens": max_tokens
        }
        
        try:
            # Send request to ChatGPT
            response = requests.post(completions_url,headers=headers,json=json_data)
            
            # Save assistant response in the dialogue
            self.dialogue.append(('assistant',response.json()['choices'][0]['text']))
            
            # Sends response back as text
            return response.json()['choices'][0]['text']
        
        except Exception as e:
            return f'Request failed with exception {e}'
        
    # Clears context and resets chat    
    def clear(self):
        self.dialogue = []
        return 'Chat context cleared'

## Limerick conversation

Here we're going to try making some rhyming poems about podcasts to get used to using our ChatGPT Class.

We'll go back and forth three times to get into the rhythm of inserting new prompts and passing context back in.

In [ ]:
# Initiate ChatGPT object and make first question
chat = ChatGPT(openai.api_key)
chat.chat('Write a rhyming limerick about podcasts in popular culture')

"\nPodcasts are now a part of our scene,\nWith stories and facts that are keen,\nThey're listened to daily,\nBy folks who are quite giddy,\nFor the entertainment they glean.<|im_end|>"

In [ ]:
# Send a second request
chat.chat('Now write another one about the negative side of podcasts in popular culture')

"\nPodcasts can be great, that's true,\nBut some folks just listen to spew,\nTheir hate and their lies,\nIt's hard to believe the size,\nOf the negativity they accrue.<|im_end|>"

In [ ]:
# Check dialogue so far in format (sender,message) and using the conversation method, which formats input as the model expects
chat.dialogue

[('user',
  '\n<|im_start|>user\nWrite a rhyming limerick about podcasts in popular culture<|im_end|>\n<|im_start|>assistant'),
 ('assistant',
  "\nPodcasts are now a part of our scene,\nWith stories and facts that are keen,\nThey're listened to daily,\nBy folks who are quite giddy,\nFor the entertainment they glean.<|im_end|>"),
 ('user',
  '\n<|im_start|>user\nNow write another one about the negative side of podcasts in popular culture<|im_end|>\n<|im_start|>assistant'),
 ('assistant',
  "\nPodcasts can be great, that's true,\nBut some folks just listen to spew,\nTheir hate and their lies,\nIt's hard to believe the size,\nOf the negativity they accrue.<|im_end|>")]

In [ ]:
# Senda third request
chat.chat('Now make one last rhyming poem about why the good outweighs the bad with podcasts')

"\nThough some podcasts may be a drag,\nWith negativity that's a snag,\nThe good ones outshine,\nWith stories that entwine,\nAnd make the listening worth the snag.\n<|im_end|>"

In [ ]:
print(chat.conversation())


<|im_start|>user
Write a rhyming limerick about podcasts in popular culture<|im_end|>
<|im_start|>assistant
Podcasts are now a part of our scene,
With stories and facts that are keen,
They're listened to daily,
By folks who are quite giddy,
For the entertainment they glean.<|im_end|>
<|im_start|>user
Now write another one about the negative side of podcasts in popular culture<|im_end|>
<|im_start|>assistant
Podcasts can be great, that's true,
But some folks just listen to spew,
Their hate and their lies,
It's hard to believe the size,
Of the negativity they accrue.<|im_end|>
<|im_start|>user
Now make one last rhyming poem about why the good outweighs the bad with podcasts<|im_end|>
<|im_start|>assistant
Though some podcasts may be a drag,
With negativity that's a snag,
The good ones outshine,
With stories that entwine,
And make the listening worth the snag.
<|im_end|>


In [ ]:
# Clear chat context and check dialogue is empty
chat.clear()
chat.dialogue

[]

## Generate fake data

Now we're going to use ChatGPT to generate some fictional data and build on the initial context that we set

In [ ]:
# Supply initial prompt
chat.chat("Generate me a bulleted list of 10 fictional customers for a GP practice. I'm a doctor who is building a database of customers to analyse")

'\n- John Smith, age 35, male, occupation: construction worker\n- Jane Doe, age 42, female, occupation: stay-at-home mom\n- Michael Johnson, age 55, male, occupation: accountant\n- Sarah Lee, age 29, female, occupation: teacher\n- David Brown, age 48, male, occupation: lawyer\n- Emily Davis, age 31, female, occupation: nurse\n- William Miller, age 62, male, occupation: retiree\n- Melissa Garcia, age 24, female, occupation: student\n- Joseph Rodriguez, age 39, male, occupation: delivery driver\n- Amanda Turner, age 50, female, occupation: business owner<|im_end|>'

In [ ]:
# I'll extract the list of names out of this response using our completions endpoint 
name_list = openai.Completion.create(engine=COMPLETIONS_MODEL,prompt=f"Return a comma-delimited list of names from this text:\n {chat.dialogue[1][1]}",max_tokens=1000)
client_names = [x.strip() for x in name_list['choices'][0]['text'].split(',')]
client_names

In [ ]:
# Lets get ChatGPT to generate me some medical history for one of these
appointment_notes = chat.chat("Generate appointment notes for John Smith's last five appointments")
print(appointment_notes)


Appointment 1:
- Date: January 5th, 2022
- Reason for visit: Follow-up for back pain
- Assessment: Patient reported that his back pain has improved with physical therapy and over-the-counter pain medication. No red flags noted on physical examination.
- Treatment: Advised to continue physical therapy and to follow up in 4 weeks.

Appointment 2:
- Date: February 2nd, 2022
- Reason for visit: Follow-up for back pain
- Assessment: Patient reported that his back pain has not improved and has increased in intensity. Physical examination revealed muscle spasms and tenderness in the lower back.
- Treatment: Prescribed muscle relaxants and referred to a specialist for further evaluation.

Appointment 3:
- Date: March 9th, 2022
- Reason for visit: Follow-up for back pain and specialist referral
- Assessment: Patient reported that muscle relaxants have helped with the pain but still experiencing discomfort. Specialist recommended an MRI and physical therapy.
- Treatment: Scheduled an MRI and ad

In [ ]:
# Lets ask ChatGPT to use this context to comment on John's trending condition
condition_analysis = chat.chat("How has John's condition been over 2022, is he improving or declining?")
print(condition_analysis)


Based on the appointment notes provided, it appears that John's condition has been declining over the course of 2022. In the first appointment, he reported that his back pain had improved with physical therapy and over-the-counter pain medication. However, in subsequent appointments, he reported that his pain had not improved and had increased in intensity. An MRI revealed a herniated disc in his lower back, and he was referred to a specialist for further evaluation. The specialist recommended surgery as an option, indicating that his condition has not improved with other treatments.
<|im_end|>


In [ ]:
# Now lets call back to the start of the conversation and ask it to cluster our patients and come up with some next steps
clustering = chat.chat("Please cluster my 10 patients into meaningful categories based on their conditions and suggest next steps for each")
print(clustering)


1. Musculoskeletal conditions:
- John Smith (back pain)
- Michael Johnson (knee pain)
- David Brown (shoulder pain)
- Emily Davis (neck pain)
- William Miller (hip pain)

Next steps: Referral to a physical therapist for assessment and treatment, and follow-up appointments to monitor progress. Consideration of imaging (such as X-rays or MRI) if necessary.

2. Cardiovascular conditions:
- Jane Doe (high blood pressure)
- Sarah Lee (high cholesterol)
- Melissa Garcia (family history of heart disease)

Next steps: Referral to a cardiologist for assessment and treatment, regular monitoring of blood pressure and cholesterol levels, and lifestyle changes (such as diet and exercise) to improve cardiovascular health.

3. Respiratory conditions:
- Michael Johnson (asthma)
- David Brown (chronic bronchitis)

Next steps: Referral to a pulmonologist for assessment and treatment, regular monitoring of symptoms, and use of inhaled medications as needed.

4. Mental health conditions:
- Sarah Lee (anx

In [ ]:
# Clear chat context and check dialogue is empty
chat.clear()
chat.dialogue

[]

### Summary

Very cool - we've now got a ChatGPT helper that we can use to maintain inputs and context. We look forward to seeing what you build with this!